In [5]:
import cv2
import numpy as np
import os

In [6]:
files = os.listdir("Data/")
print(files)

['damage168.MOV', 'damage84.MOV', 'nodamage.MOV']


In [8]:
Data_path = "Data/"
video_path = Data_path + files[1]
print(video_path)

Data/damage84.MOV


In [10]:
cap = cv2.VideoCapture(video_path)

#video information
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = int(cap.get(cv2.CAP_PROP_))

print(frameCount)

43466
